<a href="https://colab.research.google.com/github/squeeko/TensorflowProjects/blob/master/TF_TxtClasswTF_Hub_MVE_RVWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification with TensorFlow Hub: Movie reviews

This notebook classifies movie reviews as positive or negative using the text of the review. This is an example of binary—or two-class—classification, an important and widely applicable kind of machine learning problem.

The tutorial demonstrates the basic application of transfer learning with TensorFlow Hub and Keras.

We'll use the [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) that contains the text of 50,000 movie reviews from the Internet Movie Database. These are split into 25,000 reviews for training and 25,000 reviews for testing. The training and testing sets are balanced, meaning they contain an equal number of positive and negative reviews.

This notebook uses [tf.keras,](https://www.tensorflow.org/guide/keras) a high-level API to build and train models in TensorFlow, and [TensorFlow Hub,](https://www.tensorflow.org/hub) a library and platform for transfer learning. For a more advanced text classification tutorial using [tf.keras,](https://www.tensorflow.org/api_docs/python/tf/keras) see the [MLCC Text Classification Guide.](https://developers.google.com/machine-learning/guides/text-classification/)

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
# pip w/ the "q" option is a "quiet" install

!pip install -q tensorflow-hub
!pip install -q tensorflow-datasets

In [4]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [5]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)

# Ensure to change the runtime to "GPU" in the Hardware Accelerator
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE" )

Version:  2.3.0
Eager mode:  True
Hub version:  0.9.0
GPU is available


## Download the IMDB dataset

The IMDB dataset is available on [imdb reviews](https://www.tensorflow.org/datasets/catalog/imdb_reviews) or on [TensorFlow datasets.](https://www.tensorflow.org/datasets) The following code downloads the IMDB dataset to your machine (or the colab runtime):


In [6]:
# Split the training set into 60% and 40%, so we'll end up with 15K examples 
# for training, 10K examples for validation and 25K examples for testing.

train_data, validation_data, test_data = tfds.load(
    name = "imdb_reviews",
    split = ('train[:60%]', 'train[60%:]', 'test'),
    as_supervised = True)


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLE4FW6/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLE4FW6/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLE4FW6/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Explore the data
Let's take a moment to understand the format of the data. Each example is a sentence representing the movie review and a corresponding label. The sentence is not preprocessed in any way. The label is an integer value of either 0 or 1, where 0 is a negative review, and 1 is a positive review.

Let's print first 10 examples.

In [7]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [9]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## Build the model
The neural network is created by stacking layers—this requires three main architectural decisions:

How to represent the text?

How many layers to use in the model?

How many hidden units to use for each layer?

In this example, the input data consists of sentences. The labels to predict are either 0 or 1.

One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have three advantages:



*   we don't have to worry about text preprocessing,
*   we can benefit from transfer learning,
*   the embedding has a fixed size, so it's simpler to process.


















For this example we will use a **pre-trained text embedding model** from [TensorFlow Hub](https://www.tensorflow.org/hub) called [google/tf2-preview/gnews-swivel-20dim/1.](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1)

There are three other pre-trained models to test for the sake of this tutorial:

[google/tf2-preview/gnews-swivel-20dim-with-oov/1 ](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) - same as [google/tf2-preview/gnews-swivel-20dim/1,](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) but with 2.5% vocabulary converted to OOV buckets. This can help if vocabulary of the task and vocabulary of the model don't fully overlap.

[google/tf2-preview/nnlm-en-dim50/1 ](https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1)- A much larger model with ~1M vocabulary size and 50 dimensions.

[google/tf2-preview/nnlm-en-dim128/1 ](https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1)- Even larger model with ~1M vocabulary size and 128 dimensions.

Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. Note that no matter the length of the input text, the output shape of the embeddings is: (num_examples, embedding_dimension).

## SIDEBAR - Neural Network Embeddings Explained 

[TF Word Embeddings](https://www.tensorflow.org/tutorials/text/word_embeddings)

Excerpt from Towards Data Science Medium article on Word Embeddings - [NN Embeddings](https://towardsdatascience.com/neural-network-embeddings-explained-4d028e6f0526)

An embedding is a mapping of a discrete — categorical — variable to a vector of continuous numbers. In the context of neural networks, embeddings are low-dimensional, learned continuous vector representations of discrete variables. Neural network embeddings are useful because they can reduce the dimensionality of categorical variables and meaningfully represent categories in the transformed space.

Neural network embeddings have 3 primary purposes:

Finding nearest neighbors in the embedding space. These can be used to make recommendations based on user interests or cluster categories.

As input to a machine learning model for a supervised task.

For visualization of concepts and relations between categories.

In [10]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, 
                           input_shape = [],
                           dtype = tf.string,
                           trainable = True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

Let's now build the full model:

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


The layers are stacked sequentially to build the classifier:




1.   The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. The pre-trained text embedding model that we are using  [(google/tf2-preview/gnews-swivel-20dim/1)](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: **(num_examples, embedding_dimension)**.

2.   This fixed-length output vector is piped through a fully-connected **(Dense)** layer with 16 hidden units.

3.   The last layer is densely connected with a single output node.

Let's compile the model.






## Loss function and optimizer

A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs logits (a single-unit layer with a linear activation), we'll use the binary_crossentropy loss function.

This isn't the only choice for a loss function, you could, for instance, choose mean_squared_error. But, generally, binary_crossentropy is better for dealing with probabilities—it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions.

Later, when we are exploring regression problems (say, to predict the price of a house), we will see how to use another loss function called mean squared error.

Now, configure the model to use an optimizer and a loss function:

In [13]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits =True),
              metrics=['accuracy'])

## Train the model

Train the model for 20 epochs in mini-batches of 512 samples. This is 20 iterations over all samples in the **x_train and y_train** tensors. While training, monitor the model's loss and accuracy on the 10,000 samples from the validation set:

In [14]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs = 20,
                    validation_data = validation_data.batch(512),
                    verbose = 1)

Epoch 1/20
30/30 [==============================] - 2s 77ms/step - loss: 0.8982 - accuracy: 0.4631 - val_loss: 0.8059 - val_accuracy: 0.4944
Epoch 2/20
30/30 [==============================] - 2s 72ms/step - loss: 0.7433 - accuracy: 0.5288 - val_loss: 0.6961 - val_accuracy: 0.5567
Epoch 3/20
30/30 [==============================] - 2s 71ms/step - loss: 0.6599 - accuracy: 0.5859 - val_loss: 0.6373 - val_accuracy: 0.6010
Epoch 4/20
30/30 [==============================] - 2s 74ms/step - loss: 0.6121 - accuracy: 0.6301 - val_loss: 0.5985 - val_accuracy: 0.6516
Epoch 5/20
30/30 [==============================] - 2s 73ms/step - loss: 0.5718 - accuracy: 0.6755 - val_loss: 0.5622 - val_accuracy: 0.6940
Epoch 6/20
30/30 [==============================] - 2s 74ms/step - loss: 0.5260 - accuracy: 0.7279 - val_loss: 0.5155 - val_accuracy: 0.7309
Epoch 7/20
30/30 [==============================] - 2s 71ms/step - loss: 0.4740 - accuracy: 0.7722 - val_loss: 0.4713 - val_accuracy: 0.7695
Epoch 8/20
30

## Evaluate the model
And let's see how the model performs. Two values will be returned. Loss (a number which represents our error, lower values are better), and accuracy.

In [15]:
results = model.evaluate(test_data.batch(512), verbose = 1)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 [==============================] - 2s 49ms/step - loss: 0.3168 - accuracy: 0.8648
loss: 0.317
accuracy: 0.865


This fairly naive approach achieves an accuracy of about 87%. With more advanced approaches, the model should get closer to 95%.

For a more general way to work with string inputs and for a more detailed analysis of the progress of accuracy and loss during training, see the  [Text classification with preprocessed text tutorial.](https://www.tensorflow.org/tutorials/keras/text_classification)